## 서울사고건수 LSTM 예시

In [ ]:
#필요한 라이브러리
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
tf.config.list_physical_devices()
#CPU만 작동하게 하는 명령어
tf.config.experimental.set_visible_devices([], 'GPU')

In [ ]:
#파일 불러오기
#2017~2019년도
df = pd.read_csv('./LSTM_데이터/서울사고건수(2017~2019).csv')
#아래 데이터와 2017년도가 겹치므로 18~19년도만 선택하면
df= df.iloc[365:]

#2015~2017년도
#18~19년도 데이터 형식에 맞게 전처리
df2= pd.read_excel('./LSTM_데이터/서울발생건수(2015~2017).xlsx')
df2.drop(['Unnamed: 0','발생시도'],axis=1,inplace=True)
df2.rename({'계':'사고건수'},axis=1,inplace=True)

#날짜를 datatime으로 변환하기 위해선 20XX-XX-XX 00:00:00'형식으로 맞출 필요가 있는걸로 보임
#따라서 그에 적절하게 형태를 바꿈
year_lst=[]
month_lst=[]
day_lst=[]
str_date_lst=[]

for i in range(len(df2['발생일'])):
    year_lst.append(str(df2['발생일'][i])[:4])
    month_lst.append(str(df2['발생일'][i])[4:6])
    day_lst.append(str(df2['발생일'][i])[6:])
    str_date_lst.append(year_lst[i]+'-'+month_lst[i]+'-'+day_lst[i]+' '+'00:00:00')
df2['발생일']=str_date_lst


#데이터 프레임 합치기
df= pd.concat([df2,df])
#데이터 타입변환 str->datetime으로
df['발생일']=pd.to_datetime(df['발생일'])
print(df.info())
#날짜순으로 정렬
df.sort_values(by='발생일')
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)
df


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1826 entries, 0 to 1094
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   발생일     1826 non-null   datetime64[ns]
 1   사고건수    1826 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 42.8 KB
None


,발생일,사고건수
0,2015-01-01,62
1,2015-01-02,64
2,2015-01-03,94
3,2015-01-04,86
4,2015-01-05,123
...,...,...
1821,2019-12-27,94
1822,2019-12-28,94
1823,2019-12-29,78
1824,2019-12-30,91


In [ ]:
df2= pd.read_excel('./LSTM_데이터/서울발생건수(2015~2017).xlsx')
df2.drop(['Unnamed: 0','발생시도'],axis=1,inplace=True)
df2.rename({'계':'사고건수'},axis=1,inplace=True)

#날짜를 datatime으로 변환하기 위해선 20XX-XX-XX형식으로 맞출 필요가 있는걸로 보임
#따라서 그에 적절하게 형태를 바꿈
year_lst=[]
month_lst=[]
day_lst=[]
str_date_lst=[]

for i in range(len(df2['발생일'])):
    year_lst.append(str(df2['발생일'][i])[:4])
    month_lst.append(str(df2['발생일'][i])[4:6])
    day_lst.append(str(df2['발생일'][i])[6:])
    str_date_lst.append(year_lst[i]+'-'+month_lst[i]+'-'+day_lst[i])

df2['사고건수']=str_date_lst
df2

In [ ]:
#교육 및 예측을 수행하기 전에 데이터가 어떻게 표시되는지 plotly 사용하여 시각화

import plotly
import plotly.graph_objects as go
print("Plotly Version: ",plotly.__version__)

trace = go.Scatter(
    x = df['발생일'],
    y = df['사고건수'],
    mode = 'lines',
    name = 'Data'
)
layout = go.Layout(
    title = "",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Accident"}
)
fig = go.Figure(data=[trace], layout=layout)
fig.show()

Plotly Version:  5.9.0


In [ ]:
#데이터 사전처리
#분석을 위해 처음 80%의 데이터에 대해 모델을 교육하고 나머지 20%에 대해 테스트

accident_data = df['사고건수'].values
accident_data = accident_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(accident_data))

accident_train = accident_data[:split]
accident_test = accident_data[split:]

date_train = df['발생일'][:split]
date_test = df['발생일'][split:]

print(len(accident_train))
print(len(accident_test))

1460
366


- 교육을 실시하기 전에 데이터를 크게 수정해야 함
- 우리의 데이터는 여전히 시퀀스고 neural network은 supervised model로 훈련되기 때문에 데이터를 시퀀스에서 supervised data로 변환해야 함
- 기계 학습 모델의 신경망을 훈련하려면 데이터가 {<features>,<target>} 형식이어야 함
- 마찬가지로 주어진 데이터를 이 형식으로 변환해야 함 => 룩백

## Keras의 시계열 생성기 tf.keras.TimeseriesGenerator
- 시간 순서가 있는 데이터의 배치를 생성하는 도구 클래스
- arguments로 (data,targets,lengths,sample_rate=1,stride=1,start_index=0,end_index,shuffle=False,reverse=False,
batch_size=128)를 갖는다
- arguements에 대한 설명
    - data: 리스트 또는 NumPy 배열과 같이 인덱싱 가능한 2D 데이터로 0번째 축axis은 연속된 시점에 모인 표본sample들로 이루어진 시간 차원을 나타냅니다.
    - targets: data의 시간 단계와 상응하는 목표값으로 0번째 축의 길이가 data와 서로 같아야 합니다.
    - length: 생성할 배치의 시계열 길이를 지정합니다. 해당 인자를 통해 지정되는 길이는 최대 길이로서, 각 표본의 실제 길이는 length를 sampling_rate로 나눈 몫만큼이 됩니다.
    - sampling_rate: length를 통해 지정된 시계열 범위 가운데 sampling_rate 시점마다 입력값을 추출해서 배치에 포함시킬 것인지를 정합니다. 예를 들어 표본이 i번째 데이터에서
    시작할 때 sampling_rate를 r로 설정할 경우 생성되는 표본은 data[i], data[i+r], data[i+2r]... 의 형태가 되며, 표본의 최종 길이는 length를 sampling_rate로 나눈 몫이 됩니다. 기본값은 1이며, 이 경우 배치의 길이는 length와 같아집니다.
    - stride: 입력값 가운데 stride로 지정한 순서마다 표본을 생성합니다. 예를 들어 첫번째 시계열 표본이 i번째 입력값에서 시작할 때 stride가 s면 다음 표본은 data[i+s]부터, 그
    다음 표본은 data[i+2s]부터 생성됩니다. 표본 사이에 데이터가 중복되지 않게 하려면 stride값을 length보다 같거나 크게 지정하면 됩니다. 기본값은 1입니다.
    - start_index: 입력값 가운데 배치 생성에 사용할 최초 시점을 지정합니다. start_index이전의 데이터는 사용되지 않기 때문에 별도의 시험/검증 세트를 만드는 데 활용할 수 있습니다.
    기본값은 0입니다.
    - end_index: 입력값 가운데 배치 생성에 사용할 마지막 시점을 지정합니다. end_index이후의 데이터는 사용되지 않기 때문에 별도의 시험/검증 세트를 만드는 데 활용할 수 있습니다. 기본값은
    None으로, 이 경우 입력 데이터의 가장 마지막 인덱스가 자동으로 지정됩니다.
    - shuffle: bool. True인 경우, 생성한 표본의 순서를 뒤섞습니다. 기본값은 False입니다.
    - reverse: bool. True인 경우, 생성한 표본의 순서는 입력된 데이터의 역순입니다. 기본값은 False입니다.
    - batch_size: 하나의 배치 안에 포함될 표본의 개수입니다. 기본값은 128입니다.

### 생성모델: 주어진 학습 데이터를 학습하여 학습 데이터의 분포를 따르는 유사한 데이터를 생성하는 모델

In [ ]:
#Keras의 시계열 생성기(TimeseriesGenerator) 모듈 사용
# 15일을 간격으로 설정하였음
look_back = 15


train_generator = TimeseriesGenerator(accident_train, accident_train, length=look_back, batch_size=20)
test_generator = TimeseriesGenerator(accident_test, accident_test, length=look_back, batch_size=1)

## Keras tf.keras.layers.LSTM
Arguments

- units: Positive integer, dimensionality of the output space.
- activation: Activation function to use. Default: hyperbolic tangent (tanh). If you pass None, no activation is
applied (ie. "linear" activation: a(x) = x).
- recurrent_activation: Activation function to use for the recurrent step. Default: sigmoid (sigmoid). If you pass
None, no activation is applied (ie. "linear" activation: a(x) = x).
- use_bias: Boolean (default True), whether the layer uses a bias vector.
- kernel_initializer: Initializer for the kernel weights matrix, used for the linear transformation of the inputs.
Default: glorot_uniform.
- recurrent_initializer: Initializer for the recurrent_kernel weights matrix, used for the linear transformation of
the recurrent state. Default: orthogonal.
- bias_initializer: Initializer for the bias vector. Default: zeros.
- unit_forget_bias: Boolean (default True). If True, add 1 to the bias of the forget gate at initialization. Setting
it to true will also force bias_initializer="zeros". This is recommended in Jozefowicz et al..
- kernel_regularizer: Regularizer function applied to the kernel weights matrix. Default: None.
- recurrent_regularizer: Regularizer function applied to the recurrent_kernel weights matrix. Default: None.
bias_regularizer: Regularizer function applied to the bias vector. Default: None.
- activity_regularizer: Regularizer function applied to the output of the layer (its "activation"). Default: None.
- kernel_constraint: Constraint function applied to the kernel weights matrix. Default: None.
- recurrent_constraint: Constraint function applied to the recurrent_kernel weights matrix. Default: None.
- bias_constraint: Constraint function applied to the bias vector. Default: None.
- dropout: Float between 0 and 1. Fraction of the units to drop for the linear transformation of the inputs. Default: 0.
- recurrent_dropout: Float between 0 and 1. Fraction of the units to drop for the linear transformation of the
recurrent state. Default: 0.
- return_sequences: Boolean. Whether to return the last output. in the output sequence, or the full sequence. Default:
 False.
- return_state: Boolean. Whether to return the last state in addition to the output. Default: False.
- go_backwards: Boolean (default False). If True, process the input sequence backwards and return the reversed sequence.
- stateful: Boolean (default False). If True, the last state for each sample at index i in a batch will be used as
initial state for the sample of index i in the following batch.
- time_major: The shape format of the inputs and outputs tensors. If True, the inputs and outputs will be in shape
[timesteps, batch, feature], whereas in the False case, it will be [batch, timesteps, feature]. Using time_major = True is a bit more efficient because it avoids transposes at the beginning and end of the RNN calculation. However, most TensorFlow data is batch-major, so by default this function accepts input and emits output in batch-major form.
unroll: Boolean (default False). If True, the network will be unrolled, else a symbolic loop will be used. Unrolling can speed-up a RNN, although it tends to be more memory-intensive. Unrolling is only suitable for short sequences.

In [ ]:
#Neural Network, 데이터가 준비 되었으니 네트워크를 만들고 교육할 수 있음
#loss='mse'를 많이 사용

from keras.models import Sequential
from keras.layers import LSTM, Dense, InputLayer


model = Sequential()
model.add(
    LSTM(128,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 128)               66560     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 200
model.fit_generator(train_generator, epochs=num_epochs, verbose=True)

Epoch 1/200


/var/folders/m9/0vsmjxhd7q30k5mzw0bk58hh0000gn/T/ipykernel_39190/1435346254.py:2: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



73/73 [==============================] - 1s 5ms/step - loss: 1207.4587
Epoch 2/200
73/73 [==============================] - 0s 5ms/step - loss: 5971.3501
Epoch 3/200
73/73 [==============================] - 0s 5ms/step - loss: 444.5878
Epoch 4/200
73/73 [==============================] - 0s 6ms/step - loss: 382.0402
Epoch 5/200
73/73 [==============================] - 0s 5ms/step - loss: 390.2737
Epoch 6/200
73/73 [==============================] - 0s 5ms/step - loss: 380.9594
Epoch 7/200
73/73 [==============================] - 0s 5ms/step - loss: 373.1801
Epoch 8/200
73/73 [==============================] - 0s 5ms/step - loss: 405.0671
Epoch 9/200
73/73 [==============================] - 0s 5ms/step - loss: 381.7911
Epoch 10/200
73/73 [==============================] - 0s 5ms/step - loss: 361.8181
Epoch 11/200
73/73 [==============================] - 0s 5ms/step - loss: 365.2433
Epoch 12/200
73/73 [==============================] - 0s 5ms/step - loss: 392.0200
Epoch 13/200
73/73 [===

In [ ]:
#예측(Prediction, 검정 데이터에 대해 모형을 검정하고 예측과 실제 값이 겹치는지 확인할 수 있음)
#Ground Truth : 우리가 정한 정답, 우리의 모델이 우리가 원하는 답으로 예측해주길 바라는 답

prediction = model.predict_generator(test_generator)

accident_train = accident_train.reshape((-1))
accident_test = accident_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = accident_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = accident_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "-",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Accident"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

62/73 [========================>.....] - ETA: 0s - loss: 404.4930

/var/folders/m9/0vsmjxhd7q30k5mzw0bk58hh0000gn/T/ipykernel_39190/2624718467.py:4: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



In [ ]:
prediction

array([ 98.09941 , 103.49993 , 103.987564, 107.9839  , 102.81103 ,
        65.83552 , 105.73361 , 101.88356 , 105.585884, 106.43439 ,
       111.381485, 107.63686 ,  93.28674 ,  95.136734, 106.32729 ,
       102.75062 , 113.5887  , 107.54958 , 108.069916,  70.463135,
        92.93766 ,  58.80848 ,  51.97775 ,  67.6501  ,  80.74119 ,
        89.237625,  77.42021 ,  84.78736 ,  89.5723  ,  95.11053 ,
       107.25643 , 109.441505,  96.38084 , 100.45966 , 111.07391 ,
       101.17799 ,  97.99807 , 109.41527 , 107.16383 , 106.35888 ,
       103.51627 ,  98.75956 , 104.24981 , 102.61381 , 103.34    ,
       106.180016,  99.12973 ,  97.54447 ,  89.51449 ,  90.118675,
        89.72035 ,  97.56314 , 110.779526, 110.669556,  75.47323 ,
       100.207504,  98.621765, 103.98668 , 119.485146, 113.98522 ,
       117.09232 ,  76.87108 ,  99.609726, 102.697815, 107.476685,
       118.23786 , 119.02812 , 105.84219 ,  79.83317 , 110.619545,
       108.150116, 108.59241 , 102.91425 , 112.12499 , 119.533

In [ ]:
#미래예측(Forecasting)
#내일의 값을 예측하고 싶을 경우, 지난 n일(look_back)의 값을 모델에 입력하면 내일의 값을 출력으로 얻을 수 있음
#모레 값을 얻으려면 n-1일 지난 값을 내일 값 및 모레 모델 출력 값과 함께 피드 인 하면 됨
#더 긴 기간을 예측하는 것은 실현 가능하지 않으므로 한 달 동안의 사고건수를 예측

accident_data = accident_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = accident_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['발생일'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

1/1 [==============================] - 0s 7ms/step


In [ ]:
#시각화

trace1 = go.Scatter(
    x = df['발생일'].tolist(),
    y = accident_data,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)
layout = go.Layout(
    title = "-",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Accident"}
)

fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

## 날씨 월별로 예측